In [ ]:
# STUDI KASUS SIMULASI SISTEM PIKET IT DEL

import simpy
import random
import pandas as pd

# =========================
# PARAMETER SISTEM
# =========================
TOTAL_MEJA = 60
OMPRENG_PER_MEJA = 3
TOTAL_OMPRENG = TOTAL_MEJA * OMPRENG_PER_MEJA

# Total mahasiswa piket (sesuai soal)
PETUGAS_TOTAL = 7

# Waktu proses (menit)
WAKTU_LAUK = (0.5, 1)     # 30 detik – 1 menit
WAKTU_NASI = (0.5, 1)     # 30 detik – 1 menit
WAKTU_ANGKUT = (0.33, 1)  # 20 detik – 1 menit

# Kapasitas batch angkut
BATCH_MIN = 4
BATCH_MAX = 7

# =========================
# PROSES OMPRNG
# =========================
def proses_ompreng(env, id_ompreng, petugas, buffer_angkut, hasil):
    waktu_mulai = env.now

    # Proses 1: Isi lauk
    with petugas.request() as req:
        yield req
        yield env.timeout(random.uniform(*WAKTU_LAUK))

    # Masuk ke buffer menunggu pengangkutan
    yield buffer_angkut.put(id_ompreng)

    # Menunggu sampai ompreng selesai diangkut
    yield buffer_angkut.get(lambda x: x == id_ompreng)

    # Proses 3: Isi nasi
    with petugas.request() as req:
        yield req
        yield env.timeout(random.uniform(*WAKTU_NASI))

    waktu_selesai = env.now
    hasil.append(waktu_selesai - waktu_mulai)

# =========================
# PROSES ANGKUT (BATCH)
# =========================
def proses_angkut(env, petugas, buffer_angkut):
    while True:
        ukuran_batch = random.randint(BATCH_MIN, BATCH_MAX)
        batch = []

        # Mengambil ompreng sebanyak ukuran batch
        for _ in range(ukuran_batch):
            item = yield buffer_angkut.get()
            batch.append(item)

        # Waktu angkut satu kali perjalanan
        with petugas.request() as req:
            yield req
            yield env.timeout(random.uniform(*WAKTU_ANGKUT))

        # Mengembalikan ompreng ke buffer (sudah di atas meja)
        for item in batch:
            yield buffer_angkut.put(item)

# =========================
# SIMULASI UTAMA
# =========================
def run_simulation():
    # Simulasi dimulai pukul 07.00 (420 menit)
    env = simpy.Environment(initial_time=7 * 60)

    petugas = simpy.Resource(env, capacity=PETUGAS_TOTAL)
    buffer_angkut = simpy.FilterStore(env)

    hasil = []

    # Proses angkut berjalan paralel
    env.process(proses_angkut(env, petugas, buffer_angkut))

    # Membuat seluruh ompreng
    for i in range(TOTAL_OMPRENG):
        env.process(proses_ompreng(env, i, petugas, buffer_angkut, hasil))

    env.run()
    return hasil, env.now

# =========================
# EKSEKUSI SIMULASI
# =========================
hasil, waktu_akhir = run_simulation()

jam_selesai = int(waktu_akhir // 60)
menit_selesai = int(waktu_akhir % 60)

print("Total ompreng diproses:", TOTAL_OMPRENG)
print(f"Waktu selesai simulasi: {jam_selesai:02d}:{menit_selesai:02d}")
print("Rata-rata waktu penyelesaian per ompreng (menit):", sum(hasil) / len(hasil))


Total ompreng diproses: 180
Waktu selesai simulasi: 07:39
Rata-rata waktu penyelesaian per ompreng (menit): 29.31027937285129
